# Chapter 7: Advanced OOP Concepts and Design Patterns

With foundational OOP mastered, we now explore Python's advanced object-oriented capabilities. This chapter covers descriptors (the mechanism behind properties), metaclasses (controlling class creation), dataclasses (modern boilerplate reduction), and professional design patterns. These concepts separate intermediate developers from senior engineers who can architect scalable, maintainable systems.

---

## 7.1 Descriptors: Attribute Access Control

Descriptors are Python's most powerful and underutilized OOP feature. They are objects that implement `__get__`, `__set__`, or `__delete__`, allowing you to customize attribute access. Python's `@property`, `classmethod`, and `staticmethod` are all implemented as descriptors.

### The Descriptor Protocol

```python
# advanced_oop.py
from typing import Any, Optional, overload

class Validator:
    """
    Base class for descriptor validators.
    Ensures attributes meet specific criteria.
    """
    def __init__(self, min_value: Optional[int] = None, max_value: Optional[int] = None):
        self.min_value = min_value
        self.max_value = max_value
        self.name = ""
        self.owner_name = ""
    
    def __set_name__(self, owner: type, name: str) -> None:
        """
        Called when descriptor is assigned to a class attribute.
        Python 3.6+ feature.
        """
        self.name = name
        self.owner_name = owner.__name__
    
    def __get__(self, instance: Any, owner: type) -> Any:
        """
        Called when attribute is accessed (obj.attr).
        If instance is None, return the descriptor itself (for class access).
        """
        if instance is None:
            return self
        # Store value in instance dict with mangled name to avoid collision
        return instance.__dict__.get(self.name)
    
    def __set__(self, instance: Any, value: Any) -> None:
        """
        Called when attribute is set (obj.attr = value).
        """
        self.validate(value)
        instance.__dict__[self.name] = value
    
    def validate(self, value: Any) -> None:
        """Override in subclasses."""
        pass

class Integer(Validator):
    """Descriptor ensuring attribute is an integer within bounds."""
    
    def validate(self, value: Any) -> None:
        if not isinstance(value, int):
            raise TypeError(f"{self.name} must be an int, got {type(value).__name__}")
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"{self.name} must be >= {self.min_value}")
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self.name} must be <= {self.max_value}")

class String(Validator):
    """Descriptor ensuring attribute is a string with length constraints."""
    
    def __init__(self, min_len: int = 0, max_len: int = 100):
        super().__init__()
        self.min_len = min_len
        self.max_len = max_len
    
    def validate(self, value: Any) -> None:
        if not isinstance(value, str):
            raise TypeError(f"{self.name} must be a str")
        if len(value) < self.min_len:
            raise ValueError(f"{self.name} must be at least {self.min_len} chars")
        if len(value) > self.max_len:
            raise ValueError(f"{self.name} must be at most {self.max_len} chars")

class Person:
    """Class using descriptors for validation."""
    age = Integer(min_value=0, max_value=150)
    name = String(min_len=1, max_len=50)
    
    def __init__(self, name: str, age: int) -> None:
        self.name = name  # Triggers String.__set__
        self.age = age    # Triggers Integer.__set__
    
    def __repr__(self) -> str:
        return f"Person(name='{self.name}', age={self.age})"

# Usage
person = Person("Alice", 30)
print(person)  # Person(name='Alice', age=30)

try:
    person.age = 200  # ValueError: age must be <= 150
except ValueError as e:
    print(f"Validation caught: {e}")

try:
    person.name = ""  # ValueError: name must be at least 1 chars
except ValueError as e:
    print(f"Validation caught: {e}")
```

**How descriptors work:**
1.  When you access `person.age`, Python calls `type(person).__dict__['age'].__get__(person, type(person))`
2.  When you assign `person.age = 25`, Python calls `type(person).__dict__['age'].__set__(person, 25)`
3.  Descriptors are class attributes that manage instance attributes.

### Non-Data Descriptors

If a descriptor only implements `__get__` (not `__set__`), it's a "non-data descriptor" and can be overridden by instance attributes:

```python
class MethodDescriptor:
    """Simplified demonstration of how methods work."""
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner):
        """Return bound method when accessed via instance."""
        if instance is None:
            return self
        from functools import partial
        return partial(self.func, instance)

class MyClass:
    def method(self, x):
        return x * 2
    
    # method = MethodDescriptor(method)  # Roughly equivalent to what Python does

obj = MyClass()
bound = obj.method  # Descriptor returns bound method
print(bound(5))     # 10
```

---

## 7.2 Slots: Memory Optimization

By default, Python stores instance attributes in a dictionary (`__dict__`), which has significant memory overhead. `__slots__` tells Python to use a fixed-size array instead, reducing memory usage and improving attribute access speed.

### Basic Slots Usage

```python
class PointWithDict:
    """Standard class with __dict__."""
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

class PointWithSlots:
    """Optimized class using __slots__."""
    __slots__ = ('x', 'y')  # Declare allowed attributes
    
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

# Memory comparison
import sys
p_dict = PointWithDict(1.0, 2.0)
p_slots = PointWithSlots(1.0, 2.0)

print(f"PointWithDict size: {sys.getsizeof(p_dict)} bytes")
print(f"PointWithSlots size: {sys.getsizeof(p_slots)} bytes")
# Typically 50-60% memory reduction per instance

# No __dict__ attribute in slotted class
try:
    print(p_slots.__dict__)  # AttributeError
except AttributeError:
    print("No __dict__ in slotted class")

# Cannot add new attributes
try:
    p_slots.z = 3.0  # AttributeError: 'PointWithSlots' object has no attribute 'z'
except AttributeError:
    print("Cannot add arbitrary attributes to slotted class")
```

### Advanced Slots Features

```python
class SlotsDemo:
    """
    Demonstrates advanced slots capabilities.
    """
    __slots__ = ('x', 'y', '__weakref__')  # Allow weak references
    
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

class SlotsWithDefaults:
    """
    Slots with default values (requires __init__ handling).
    """
    __slots__ = ('name', 'value', '_cache')
    
    def __init__(self, name: str = "default", value: int = 0) -> None:
        self.name = name
        self.value = value
        self._cache = None  # "Private" slot

# Inheritance with slots
class Base:
    __slots__ = ('a',)
    
    def __init__(self, a: int) -> None:
        self.a = a

class Child(Base):
    __slots__ = ('b',)  # Must declare own slots, inherits parent's
    
    def __init__(self, a: int, b: int) -> None:
        super().__init__(a)
        self.b = b

# If child doesn't define __slots__, it gets __dict__ and can add attributes
class DictChild(Base):
    pass  # No __slots__, has __dict__

c = DictChild(1)
c.arbitrary = "value"  # Works
```

**When to use slots:**
*   Creating millions of instances (data processing, games, simulations)
*   When attribute set is known and fixed (data structures)
*   When you want to prevent arbitrary attribute assignment (bug prevention)

**When NOT to use slots:**
*   Need dynamic attribute assignment
*   Using multiple inheritance with slotted classes (complex)
*   Need to pickle objects with complex state (requires special handling)

---

## 7.3 Metaclasses: Classes That Create Classes

In Python, everything is an object, including classes. Classes are instances of metaclasses (by default, `type`). Metaclasses allow you to customize class creation itself—modifying attributes, enforcing naming conventions, or registering subclasses automatically.

### Basic Metaclass

```python
class Meta(type):
    """
    Metaclass that modifies class creation.
    Inherits from 'type' (the default metaclass).
    """
    
    def __new__(mcs, name: str, bases: tuple, namespace: dict, **kwargs):
        """
        Called when class is created (before __init__).
        Controls the actual class object creation.
        """
        print(f"Creating class {name}...")
        
        # Enforce naming convention: class names must be PascalCase
        if not name[0].isupper():
            raise ValueError(f"Class name {name} must start with uppercase")
        
        # Automatically convert methods to uppercase (silly example)
        for attr_name, attr_value in list(namespace.items()):
            if callable(attr_value) and not attr_name.startswith('__'):
                namespace[attr_name] = mcs._wrap_method(attr_value)
        
        # Create the class using type.__new__
        cls = super().__new__(mcs, name, bases, namespace)
        return cls
    
    @staticmethod
    def _wrap_method(func):
        """Decorator applied to all methods."""
        def wrapper(*args, **kwargs):
            print(f"Calling {func.__name__}...")
            return func(*args, **kwargs)
        return wrapper
    
    def __init__(cls, name: str, bases: tuple, namespace: dict, **kwargs):
        """
        Called after class is created.
        cls is the newly created class.
        """
        super().__init__(name, bases, namespace)
        # Add class-level attribute automatically
        cls.created_by_metaclass = True

# Using the metaclass
class MyClass(metaclass=Meta):
    def method1(self):
        return "Hello"
    
    def method2(self, x: int) -> int:
        return x * 2

obj = MyClass()
print(obj.method1())  # Prints "Calling method1..." then "Hello"
print(MyClass.created_by_metaclass)  # True
```

### Practical Metaclass: Registry Pattern

```python
class PluginRegistry(type):
    """
    Automatically registers subclasses in a central registry.
    Useful for plugin systems, ORMs, command handlers.
    """
    registry: dict[str, type] = {}
    
    def __new__(mcs, name: str, bases: tuple, namespace: dict):
        cls = super().__new__(mcs, name, bases, namespace)
        # Don't register the base class itself
        if bases:
            mcs.registry[name] = cls
            print(f"Registered plugin: {name}")
        return cls

class Plugin(metaclass=PluginRegistry):
    """Base class for all plugins."""
    def execute(self) -> None:
        raise NotImplementedError

class EmailPlugin(Plugin):
    def execute(self) -> None:
        print("Sending email...")

class SMSPlugin(Plugin):
    def execute(self) -> None:
        print("Sending SMS...")

# Access registered plugins
print(f"Available plugins: {list(PluginRegistry.registry.keys())}")
# ['EmailPlugin', 'SMSPlugin']

# Instantiate by name
plugin_name = "EmailPlugin"
plugin_class = PluginRegistry.registry[plugin_name]
instance = plugin_class()
instance.execute()
```

### Metaclass vs Class Decorators

For many use cases, class decorators are simpler than metaclasses:

```python
def singleton(cls):
    """
    Class decorator implementing singleton pattern.
    Often clearer than metaclass for simple cases.
    """
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Database:
    def __init__(self):
        print("Initializing database...")
        self.connection = "Connected"

db1 = Database()
db2 = Database()
print(db1 is db2)  # True
```

> **Guideline**: Use metaclasses only when you need to control class creation (before instantiation). For modifying already-created classes, use class decorators or `__init_subclass__` (Python 3.6+).

---

## 7.4 Dataclasses: Modern Data Containers

Introduced in Python 3.7, the `@dataclass` decorator reduces boilerplate for classes that primarily store data, automatically generating `__init__`, `__repr__`, `__eq__`, and more.

### Basic Dataclass

```python
from dataclasses import dataclass, field, asdict, astuple
from typing import List, Optional

@dataclass
class InventoryItem:
    """
    Automatically generates:
    - __init__(self, name: str, unit_price: float, quantity: int = 0)
    - __repr__ showing all fields
    - __eq__ comparing all fields
    """
    name: str
    unit_price: float
    quantity: int = 0  # Default value allowed
    
    def total_cost(self) -> float:
        return self.unit_price * self.quantity

# Usage
item = InventoryItem("Widget", 10.5, 100)
print(item)  # InventoryItem(name='Widget', unit_price=10.5, quantity=100)
print(asdict(item))  # {'name': 'Widget', 'unit_price': 10.5, 'quantity': 100}

item2 = InventoryItem("Widget", 10.5, 100)
print(item == item2)  # True (auto-generated __eq__)
```

### Advanced Dataclass Options

```python
@dataclass(frozen=True)  # Immutable instances (like NamedTuple but more flexible)
class ImmutablePoint:
    x: float
    y: float
    
    def move(self, dx: float, dy: float) -> "ImmutablePoint":
        """Returns new instance instead of modifying (functional style)."""
        return ImmutablePoint(self.x + dx, self.y + dy)

@dataclass(order=True)  # Generates __lt__, __le__, __gt__, __ge__
class PriorityItem:
    """
    order=True sorts by all fields in declaration order.
    Use field(compare=False) to exclude fields from comparison.
    """
    priority: int
    name: str = field(compare=False)  # Only sort by priority, not name
    description: Optional[str] = field(default=None, compare=False, repr=False)

items = [
    PriorityItem(2, "Task B"),
    PriorityItem(1, "Task A"),
    PriorityItem(3, "Task C")
]
print(sorted(items))  # Sorted by priority

@dataclass
class Config:
    """Demonstrates field options."""
    name: str = field(default="default_name")
    items: List[str] = field(default_factory=list)  # Mutable default workaround!
    secret: str = field(repr=False, compare=False)  # Hide in repr, exclude from eq
    
    def __post_init__(self):
        """
        Called after __init__.
        Use for validation or computed fields.
        """
        if len(self.name) < 3:
            raise ValueError("Name must be at least 3 characters")

config = Config(secret="password123")
print(config)  # Config(name='default_name', items=[], secret=...)
```

### Dataclass vs NamedTuple vs Regular Class

| Feature | NamedTuple | @dataclass | Regular Class |
|---------|------------|------------|---------------|
| Immutable | Yes | Optional | Optional |
| Memory | Efficient | Normal | Normal |
| Slots | Automatic | Optional (`slots=True`) | Manual |
| Methods | Supported | Supported | Supported |
| Inheritance | Limited | Full | Full |
| Pattern Matching | Yes | Yes (Python 3.10+) | Yes |

---

## 7.5 Context Managers: Resource Management Protocol

Context managers ensure proper acquisition and release of resources (files, locks, connections) using the `with` statement. They implement `__enter__` and `__exit__`.

### Class-Based Context Manager

```python
from typing import Optional, Type, TracebackType
import time

class Timer:
    """
    Context manager for timing code blocks.
    """
    def __init__(self, name: str = "Operation") -> None:
        self.name = name
        self.start_time: Optional[float] = None
        self.elapsed: float = 0.0
    
    def __enter__(self) -> "Timer":
        """Called when entering 'with' block."""
        self.start_time = time.perf_counter()
        return self
    
    def __exit__(
        self,
        exc_type: Optional[Type[BaseException]],
        exc_val: Optional[BaseException],
        exc_tb: Optional[TracebackType]
    ) -> Optional[bool]:
        """
        Called when exiting 'with' block (even if exception occurred).
        
        Args:
            exc_type: Exception class if exception occurred, else None
            exc_val: Exception instance if occurred, else None
            exc_tb: Traceback if exception occurred, else None
        
        Returns:
            True to suppress exception, False/None to propagate
        """
        self.elapsed = time.perf_counter() - self.start_time
        print(f"{self.name} took {self.elapsed:.4f} seconds")
        
        # Return False to propagate any exception that occurred
        return None

# Usage
with Timer("Heavy computation"):
    sum(range(1000000))

# As decorator (contextlib.contextmanager is easier for this)
```

### contextlib: Functional Context Managers

For simple cases, use `contextlib.contextmanager` (generator-based):

```python
from contextlib import contextmanager
from typing import Generator
import tempfile
import os

@contextmanager
def temporary_file(suffix: str = ".txt") -> Generator[str, None, None]:
    """
    Creates temp file, yields path, cleans up on exit.
    """
    fd, path = tempfile.mkstemp(suffix=suffix)
    try:
        print(f"Created temp file: {path}")
        yield path  # Value bound to 'as' variable
    finally:
        os.close(fd)
        os.unlink(path)
        print(f"Cleaned up: {path}")

# Usage
with temporary_file() as filepath:
    with open(filepath, 'w') as f:
        f.write("Temporary data")
    # File automatically deleted after block

@contextmanager
def database_transaction(connection):
    """
    Database transaction context manager.
    Commits on success, rolls back on exception.
    """
    try:
        yield connection
        connection.commit()
        print("Transaction committed")
    except Exception as e:
        connection.rollback()
        print(f"Transaction rolled back: {e}")
        raise

# Suppressing exceptions (Python 3.4+)
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove("maybe_missing_file.txt")  # No exception if file doesn't exist
```

### Async Context Managers (Python 3.5+)

```python
from contextlib import asynccontextmanager
from typing import AsyncGenerator

@asynccontextmanager
async def async_database_connection(host: str) -> AsyncGenerator["Connection", None]:
    """Async context manager for database connections."""
    conn = await create_connection(host)
    try:
        yield conn
    finally:
        await conn.close()

# Usage
async def fetch_data():
    async with async_database_connection("localhost") as conn:
        return await conn.query("SELECT * FROM users")
```

---

## 7.6 Design Patterns: Pythonic Implementations

Design patterns are reusable solutions to common problems. Python's dynamic nature often allows simpler implementations than Java/C++.

### Singleton Pattern (One Instance)

```python
from functools import lru_cache

@lru_cache(maxsize=None)  # Simplest singleton
class Singleton:
    def __init__(self):
        print("Initializing singleton...")
        self.value = 42

# Or using __new__
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Logger(metaclass=SingletonMeta):
    def __init__(self):
        self.logs = []
    
    def log(self, msg: str) -> None:
        self.logs.append(msg)

logger1 = Logger()
logger2 = Logger()
print(logger1 is logger2)  # True
```

### Factory Pattern (Creation Abstraction)

```python
from typing import Dict, Type, Callable
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def speak(self) -> str:
        pass

class Dog(Animal):
    def speak(self) -> str:
        return "Woof!"

class Cat(Animal):
    def speak(self) -> str:
        return "Meow!"

class AnimalFactory:
    """Factory using registry pattern."""
    _registry: Dict[str, Type[Animal]] = {}
    
    @classmethod
    def register(cls, name: str, animal_class: Type[Animal]) -> None:
        cls._registry[name] = animal_class
    
    @classmethod
    def create(cls, name: str) -> Animal:
        if name not in cls._registry:
            raise ValueError(f"Unknown animal: {name}")
        return cls._registry[name]()

# Register animals
AnimalFactory.register("dog", Dog)
AnimalFactory.register("cat", Cat)

# Usage
pet = AnimalFactory.create("dog")
print(pet.speak())  # Woof!
```

### Observer Pattern (Event Subscription)

```python
from typing import List, Callable, Any

class EventManager:
    """
    Observer pattern: objects subscribe to events,
    get notified when events occur.
    """
    def __init__(self):
        self._listeners: Dict[str, List[Callable[..., Any]]] = {}
    
    def subscribe(self, event_type: str, listener: Callable[..., Any]) -> None:
        if event_type not in self._listeners:
            self._listeners[event_type] = []
        self._listeners[event_type].append(listener)
    
    def unsubscribe(self, event_type: str, listener: Callable[..., Any]) -> None:
        self._listeners[event_type].remove(listener)
    
    def notify(self, event_type: str, *args: Any, **kwargs: Any) -> None:
        for listener in self._listeners.get(event_type, []):
            listener(*args, **kwargs)

# Usage
class DataProcessor:
    def __init__(self, event_manager: EventManager):
        self.event_manager = event_manager
        self.processed = 0
    
    def process(self, data: str) -> None:
        print(f"Processing: {data}")
        self.processed += 1
        self.event_manager.notify("data_processed", data, self.processed)

def log_handler(data: str, count: int) -> None:
    print(f"[LOG] Processed item #{count}: {data}")

def metrics_handler(data: str, count: int) -> None:
    print(f"[METRICS] Total processed: {count}")

events = EventManager()
events.subscribe("data_processed", log_handler)
events.subscribe("data_processed", metrics_handler)

processor = DataProcessor(events)
processor.process("File A")
processor.process("File B")
```

### Strategy Pattern (Interchangeable Algorithms)

```python
from typing import Protocol
from abc import ABC, abstractmethod

# Using Protocol (structural subtyping)
class PaymentStrategy(Protocol):
    def pay(self, amount: float) -> bool:
        ...

class CreditCardPayment:
    def __init__(self, card_number: str):
        self.card_number = card_number
    
    def pay(self, amount: float) -> bool:
        print(f"Charging ${amount} to card {self.card_number[-4:]}")
        return True

class PayPalPayment:
    def __init__(self, email: str):
        self.email = email
    
    def pay(self, amount: float) -> bool:
        print(f"Charging ${amount} to PayPal account {self.email}")
        return True

class ShoppingCart:
    def __init__(self, strategy: PaymentStrategy):
        self.strategy = strategy
        self.items: List[tuple[str, float]] = []
    
    def add_item(self, name: str, price: float) -> None:
        self.items.append((name, price))
    
    def checkout(self) -> bool:
        total = sum(price for _, price in self.items)
        return self.strategy.pay(total)

# Usage
cart = ShoppingCart(CreditCardPayment("1234-5678-9012-3456"))
cart.add_item("Laptop", 999.99)
cart.checkout()

cart = ShoppingCart(PayPalPayment("user@example.com"))
cart.add_item("Book", 29.99)
cart.checkout()
```

---

## 7.7 SOLID Principles in Python

SOLID principles guide maintainable, scalable OOP design.

### Single Responsibility Principle (SRP)

A class should have one reason to change.

```python
# Bad: Class does too much
class UserManagerBad:
    def save_to_database(self, user): ...
    def send_email(self, user, message): ...
    def generate_report(self, user): ...

# Good: Separated concerns
class UserRepository:
    def save(self, user): ...

class EmailService:
    def send(self, to: str, message: str): ...

class ReportGenerator:
    def generate(self, user): ...

class UserService:
    """Coordinates the specialized services."""
    def __init__(self, repo: UserRepository, email: EmailService):
        self.repo = repo
        self.email = email
```

### Open/Closed Principle (OCP)

Open for extension, closed for modification.

```python
from abc import ABC, abstractmethod
from typing import List

class DiscountStrategy(ABC):
    @abstractmethod
    def calculate(self, price: float) -> float:
        pass

class RegularDiscount(DiscountStrategy):
    def calculate(self, price: float) -> float:
        return price * 0.9  # 10% off

class PremiumDiscount(DiscountStrategy):
    def calculate(self, price: float) -> float:
        return price * 0.8  # 20% off

class PriceCalculator:
    """Extensible: add new strategies without modifying this class."""
    def __init__(self, strategy: DiscountStrategy):
        self.strategy = strategy
    
    def get_final_price(self, price: float) -> float:
        return self.strategy.calculate(price)
```

### Liskov Substitution Principle (LSP)

Subtypes must be substitutable for base types.

```python
class Rectangle:
    def __init__(self, width: float, height: float):
        self._width = width
        self._height = height
    
    @property
    def width(self) -> float:
        return self._width
    
    @width.setter
    def width(self, value: float) -> None:
        self._width = value
    
    @property
    def height(self) -> float:
        return self._height
    
    @height.setter
    def height(self, value: float) -> None:
        self._height = value
    
    def area(self) -> float:
        return self._width * self._height

class Square(Rectangle):
    """
    Violates LSP: setting width changes height, breaking Rectangle contract.
    
    Better approach: Square is not a Rectangle in OOP sense, 
    or make immutable (no setters).
    """
    def __init__(self, side: float):
        super().__init__(side, side)
    
    @Rectangle.width.setter
    def width(self, value: float) -> None:
        self._width = value
        self._height = value  # Violates LSP expectation
    
    @Rectangle.height.setter
    def height(self, value: float) -> None:
        self._width = value
        self._height = value
```

### Interface Segregation Principle (ISP)

Clients shouldn't depend on methods they don't use.

```python
from typing import Protocol

# Bad: Fat interface
class Machine(Protocol):
    def print(self, document: str): ...
    def scan(self, document: str): ...
    def fax(self, document: str): ...

# Good: Split interfaces
class Printer(Protocol):
    def print(self, document: str) -> None: ...

class Scanner(Protocol):
    def scan(self, document: str) -> str: ...

class MultiFunctionDevice(Printer, Scanner):
    """Implements only what's needed."""
    pass

class SimplePrinter:
    """Only implements Printer, not forced to implement scan/fax."""
    def print(self, document: str) -> None:
        print(f"Printing: {document}")
```

### Dependency Inversion Principle (DIP)

Depend on abstractions, not concretions.

```python
from typing import Protocol

class DatabaseConnection(Protocol):
    def query(self, sql: str) -> list[dict]:
        ...

class PostgresConnection:
    def query(self, sql: str) -> list[dict]:
        # Real implementation
        return []

class MockConnection:
    def query(self, sql: str) -> list[dict]:
        # For testing
        return [{"id": 1, "name": "Test"}]

class UserRepository:
    """Depends on abstraction (Protocol), not concrete PostgresConnection."""
    def __init__(self, connection: DatabaseConnection):
        self.connection = connection
    
    def get_user(self, user_id: int) -> dict:
        return self.connection.query(f"SELECT * FROM users WHERE id={user_id}")[0]

# Can inject real or mock database
repo = UserRepository(PostgresConnection())
repo = UserRepository(MockConnection())  # For testing
```

---

## Chapter Summary

This chapter explored Python's advanced OOP capabilities and professional design patterns:

*   **Descriptors**: Implemented custom attribute access with `__get__`, `__set__`, and `__delete__`. Used `__set_name__` for automatic registration. Descriptors power Python's `@property`, `classmethod`, and `staticmethod`.
*   **Slots**: Reduced memory footprint by 50-60% using `__slots__` instead of `__dict__`, preventing arbitrary attribute assignment and improving access speed.
*   **Metaclasses**: Controlled class creation with `__new__` and `__init__`. Implemented registry patterns for plugin systems. Preferred class decorators for simple cases.
*   **Dataclasses**: Reduced boilerplate with `@dataclass`, using `field()` for defaults and `__post_init__` for validation. Compared tradeoffs with NamedTuple and regular classes.
*   **Context Managers**: Managed resources with `__enter__`/`__exit__` and `@contextmanager`. Handled exceptions in cleanup code and implemented async context managers.
*   **Design Patterns**: Implemented Singleton (with caution), Factory, Observer, and Strategy patterns using Python's dynamic features and Protocols.
*   **SOLID Principles**: Applied SRP (single responsibility), OCP (extension), LSP (substitution), ISP (interface segregation), and DIP (dependency inversion) to create maintainable architectures.

**Key Takeaways:**
1.  Use `__slots__` for memory-critical applications with fixed schemas.
2.  Prefer dataclasses over NamedTuple for mutable data containers with inheritance needs.
3.  Avoid metaclasses unless controlling class creation is absolutely necessary; use `__init_subclass__` or class decorators instead.
4.  Context managers ensure cleanup happens even with exceptions; always use them for resources.
5.  Design patterns are guidelines, not dogma; Python's first-class functions often replace boilerplate patterns.
6.  SOLID principles prevent technical debt; DIP via Protocols enables testable, decoupled code.

---

## Preview: Chapter 8 - Code Style, Type Hints, and Quality Assurance

With advanced OOP mastered, we turn to professional code quality. In **Chapter 8: Code Style, Type Hints, and Quality Assurance**, we will cover:

*   **PEP 8 Deep Dive**: Comprehensive style guidelines, naming conventions (snake_case vs PascalCase), line length, imports organization, and automated formatting with Black.
*   **Advanced Type Hints**: Generics (`TypeVar`, `Generic`), Union types (`|`), Optional, Callable, Protocols for structural subtyping, and the `typing` module's modern replacements (e.g., `list[int]` vs `List[int]`).
*   **Static Analysis**: Using `mypy` for type checking, `pylint`/`flake8` for linting, and configuring `pyproject.toml` for tool integration.
*   **Documentation**: Docstring formats (Google, NumPy, Sphinx), type stub files (`.pyi`), and automated documentation generation.
*   **Testing Fundamentals**: Unit testing with `unittest` and `pytest`, fixtures, parametrization, mocking with `unittest.mock`, and coverage analysis.

We will configure a professional Python project from scratch, establishing the tooling and conventions used in industry-standard development workflows, preparing you for the error handling and file I/O operations in Chapter 9.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='6. classes_and_objects.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. professional_development_practices/8. code_style_and_quality.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
